In [2]:
import pandas as pd
from sqlalchemy import create_engine

## Read Raw Data

In [3]:
# Read csv file
df = pd.read_csv("../Data/clean_data/clean_data.csv")
df.head()


area  district  victim_age victim_gender victim_race  \
0  N Hollywood      1591        99.0          Male           O   
1     Foothill      1677        45.0          Male           W   
2  West Valley      1028        25.0          Male           A   
3    Northeast      1153        29.0          Male           O   
4      Central       192        41.0          Male           O   

               location             datetime  
0  (34.1371, -118.4062)  2019-07-06 23:55:00  
1  (34.2249, -118.3617)  2019-07-06 05:00:00  
2  (34.2012, -118.4989)  2019-07-06 11:30:00  
3  (34.1081, -118.3078)  2019-07-06 14:15:00  
4  (34.0384, -118.2646)  2019-07-06 12:30:00

In [25]:
df = df.rename(columns ={'crash_date_time':'datetime'})

## Normalize Data 

### 1.  Area Table

In [3]:
# Select Area from Raw Table
df_area = df.copy()
df_area = pd.DataFrame(df_area[['area','district']]).drop_duplicates().reset_index()
df_area = df_area.drop(df_area.columns[0],axis=1)
df_area

area  district
0     N Hollywood      1591
1        Foothill      1677
2     West Valley      1028
3       Northeast      1153
4         Central       192
5       Hollywood       645
6          Newton      1351
7     N Hollywood      1547
8         Topanga      2157
9         Rampart       231
10     Hollenbeck       423
11    West Valley      1088
12        Mission      1964
13        Pacific      1437
14      Hollywood       643
15      Southwest       351
16    West Valley      1025
17         Newton      1307
18         Newton      1303
19       Van Nuys       963
20       Wilshire       785
21      Hollywood       666
22    West Valley      1005
23       Van Nuys       952
24    N Hollywood      1523
25    West Valley      1011
26        Central       159
27     Hollenbeck       421
28      Northeast      1132
29      Northeast      1135
...           ...       ...
1290      Rampart       214
1291   Devonshire      1726
1292  West Valley      1059
1293   Devonshire      1736
1294      Mission      1986
1295      Rampart       254
1296  West Valley      1055
1297    Hollywood       600
1298     Foothill      1671
1299     Wilshire       793
1300     Foothill      1697
1301     Wilshire       794
1302     Wilshire       792
1303      Rampart       286
1304     Van Nuys       967
1305  West Valley      1087
1306  West Valley      1013
1307    Hollywood       674
1308      Rampart       213
1309      Rampart       277
1310      Rampart       234
1311      Rampart       296
1312      Mission      1992
1313     Wilshire       795
1314      Mission      1997
1315     Foothill      1647
1316  West Valley      1095
1317     Foothill      1646
1318   Devonshire      1706
1319     Van Nuys       962

[1320 rows x 2 columns]

### 2. Gender Table

In [4]:
# Select Gender from Raw Table
df_gender = pd.DataFrame(df['victim_gender']).drop_duplicates().reset_index(drop=True)
df_gender=df_gender.rename(columns={'victim_gender':'gender'})
df_gender

gender
0     Male
1   Female
2  Unknown

### 3. Ethnicity Table

In [16]:
# Select Race from Raw Table
df_race = pd.DataFrame(df['victim_race'])
race_dict = {'H':'Hispanic', 'B':'Black', 'O':'Unknown', 'W':'White', 'X':'Unknown', '-':'Unknown',
             'A':'Asian', 'K':'Asian', 'C':'Asian', 'F':'Asian', 'U':'Pacific Islander',
             'J':'Asian', 'P':'Pacific Islander', 'V':'Asian', 'Z':'Asian',
             'I':'American Indian', 'G':'Pacific Islander', 'S':'Pacific Islander', 'D':'Asian', 'L':'Asian'}

df_race["race_description"] = df_race["victim_race"].map(race_dict)
df_race=df_race.drop_duplicates().reset_index(drop = True)
df_race=df_race.rename(columns={'victim_race':'race_init'})
df_race=df_race[:-1]

### 4. Crash Table

In [9]:
# Select Gender from Raw Table
df_crash = pd.DataFrame(df[['location','datetime']])
df_crash = df_crash.rename(columns = {'datetime':'crash_date_time','location':'crash_location'})
df_crash.head()

crash_location crash_date_time
0  (34.1371  -118.4062)  7/6/2019 23:55
1  (34.2249  -118.3617)   7/6/2019 5:00
2  (34.2012  -118.4989)  7/6/2019 11:30
3  (34.1081  -118.3078)  7/6/2019 14:15
4  (34.0384  -118.2646)  7/6/2019 12:30

### 5. Victim Table

In [ ]:
# create victim table
df['victim_age'] = df['victim_age'].apply(int)
df_victim = pd.DataFrame(df[['victim_age']])
df_victim=df_victim.rename(columns={'victim_age':'age'})
df_victim.head()

## Export CSV

In [ ]:
# df_victim.to_csv("data/clean_data/Victim.csv",index=False)
# df_crash.to_csv("data/clean_data/Crash.csv",index=False)
# df_race.to_csv("data/clean_data/Ethnicity.csv",index=False)
# df_gender.to_csv("data/clean_data/Gender.csv",index=False)
# df_area.to_csv("data/clean_data/Area.csv",index=False)

## Create Database Connection and Load

In [10]:
connection = "postgres:amairany25@localhost:5432/crash_db"
engine = create_engine(f'postgresql://{connection}')

In [13]:
engine.table_names()

['race', 'CrashRaw']

In [27]:
# Load the data to database
# df_area.to_sql(name='Area', con=engine, if_exists='append', index=False)
# df_victim.to_sql(name='Victim', con=engine, if_exists='append', index=False)
# df_crash.to_sql(name='Crash', con=engine, if_exists='append', index=False)
# df_gender.to_sql(name='Gender', con=engine, if_exists='append', index=False)
df_race.to_sql(name='race', con=engine, if_exists='append', index=False)
df.to_sql(name='CrashRaw', con=engine, if_exists='append', index=False)